In [1]:
import os
import pandas as pd
from thefuzz import fuzz, process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [3]:
diretorio = r'C:/Users/cibelly.souza/Desktop/ANÁLISES PRICING/ESTUDOS/PROMO'
os.makedirs(diretorio, exist_ok=True)

In [5]:
#Diretório da base infomarket
diretorio = r'C:/Users/cibelly.souza/Desktop/ANÁLISES PRICING/ESTUDOS/PROMO'
arquivo = os.path.join(diretorio, 'base_infomarket_2025_05_12.csv')

In [7]:
print(os.listdir(diretorio))

['.ipynb_checkpoints', 'Acordos_Comerciais-16036-13-05-2025-10_18.csv', 'base_infomarket_2025_05_12.csv', 'Cadastro.csv', 'tabela_unificada.csv', 'Tentativa de limpeza e mesclagem de tabelas .ipynb']


In [9]:
# Carregar as tabelas
tabela1 = pd.read_csv(arquivo, sep=None, engine='python', encoding='latin1')

In [10]:
tabela2 = pd.read_csv(os.path.join(diretorio,'Acordos_Comerciais-16036-13-05-2025-10_18.csv'), delimiter=';', encoding='latin1')

In [13]:
tabela3 = pd.read_csv(os.path.join(diretorio,'Cadastro.csv'), delimiter=';', encoding='latin1')

In [14]:
#Verificando a estrutura das tabelas
print(tabela1.head())

                 leaflet_id            network           segment  \
0  6814c03fbe2e92604a52f154  BRASIL ATACADISTA  VAREJO ALIMENTAR   
1  6814c03fbe2e92604a52f154  BRASIL ATACADISTA  VAREJO ALIMENTAR   
2  6814c03fbe2e92604a52f154  BRASIL ATACADISTA  VAREJO ALIMENTAR   
3  6814c03fbe2e92604a52f154  BRASIL ATACADISTA  VAREJO ALIMENTAR   
4  6814c03fbe2e92604a52f154  BRASIL ATACADISTA  VAREJO ALIMENTAR   

            state            city leaflet_code  leaflet_name leaflet_type  \
0  Santa Catarina        Blumenau       52f154  ADEGA BRASIL  ABASTECEDOR   
1  Santa Catarina  FlorianÃ³polis       52f154  ADEGA BRASIL  ABASTECEDOR   
2  Santa Catarina  FlorianÃ³polis       52f154  ADEGA BRASIL  ABASTECEDOR   
3  Santa Catarina  FlorianÃ³polis       52f154  ADEGA BRASIL  ABASTECEDOR   
4  Santa Catarina  FlorianÃ³polis       52f154  ADEGA BRASIL  ABASTECEDOR   

   leaflet_number_of_pages leaflet_last_images_upload  ... item_unit_measure  \
0                        1   2025-05-02T12:55:58

In [17]:
print(tabela2.head())

   SEQCAMPANHA                                           CAMPANHA     MIDIA  \
0        16036  ENCARTE LOKÃO KOMPRÃO 03/05 E 04/05 (TEMA: L...  Encartes   
1        16036  ENCARTE LOKÃO KOMPRÃO 03/05 E 04/05 (TEMA: L...  Encartes   
2        16036  ENCARTE LOKÃO KOMPRÃO 03/05 E 04/05 (TEMA: L...  Encartes   
3        16036  ENCARTE LOKÃO KOMPRÃO 03/05 E 04/05 (TEMA: L...  Encartes   
4        16036  ENCARTE LOKÃO KOMPRÃO 03/05 E 04/05 (TEMA: L...  Encartes   

       INICIO         FIM  SEQLOJA  SEQOFERTA TIPO_OFERTA  SEQPRODUTO  \
0  2025-05-03  2025-05-04       10     540899      Normal        9218   
1  2025-05-03  2025-05-04       10     540900      Normal       11227   
2  2025-05-03  2025-05-04       10     540901      Normal       40601   
3  2025-05-03  2025-05-04       10     540902   Club Koch       63195   
4  2025-05-03  2025-05-04       10     540903      Normal       21836   

                                  DESCRICAO  ...  NEGOCIACAO_ESPACO  \
0              

In [19]:
print(tabela3.head())

   ï»¿EMPRESA  COD_PRODUTO    cidade      negocio COD_BARRAS  COD_FAMILIA  \
0          50        77957  Araquari  2.ATACAREJO        102        75788   
1          50        17058  Araquari  2.ATACAREJO        130        12243   
2          50         8313  Araquari  2.ATACAREJO        260         5962   
3          50        11856  Araquari  2.ATACAREJO        277         8521   
4          50        22463  Araquari  2.ATACAREJO        437        16058   

                    DSC_PRODUTO      NIVEL1        NIVEL2              NIVEL3  \
0  FAR MILHO DALON MEDIA PC 1KG   MERCEARIA  SECA SALGADA  GRAOS E FARINACEOS   
1                     SALSAO UN  PERECIVEIS    HORTIFRUTI            VERDURAS   
2            ABOBORA CABOTIA KG  PERECIVEIS    HORTIFRUTI             LEGUMES   
3               GALINHA CONG KG  PERECIVEIS       ACOUGUE                 AVE   
4      RICOTA LASAROLI S/SAL KG  PERECIVEIS           PAS           RESFRIADO   

                   NIVEL4         NIVEL5   NIVEL6 

In [21]:
print(tabela1.shape[1],tabela2.shape[1],tabela3.shape[1])

36 25 14


In [23]:
print(tabela1.columns)

Index(['leaflet_id', 'network', 'segment', 'state', 'city', 'leaflet_code',
       'leaflet_name', 'leaflet_type', 'leaflet_number_of_pages',
       'leaflet_last_images_upload', 'leaflet_created', 'leaflet_updated',
       'validity_start_date', 'validity_finish_date', 'period',
       'item_description', 'ean', 'price', 'minimum_quantity', 'dynamic',
       'requires_loyality', 'requires_credit_card', 'wholesale_only',
       'price_created', 'price_updated', 'item_measure', 'item_unit_measure',
       'category', 'supercategory', 'division', 'brand', 'manufacturer',
       'category_id', 'supercategory_id', 'images_url', 'channel_name'],
      dtype='object')


In [25]:
#Reduzindo base Infomarket para apenas o necessário (selecionando apenas as colunas nessa posição)
tabela1 = tabela1[['network','city','leaflet_name','leaflet_type','leaflet_number_of_pages','validity_start_date', 'validity_finish_date', 'period',
                                'item_description', 'ean', 'price', 'minimum_quantity', 'dynamic','item_measure', 'item_unit_measure','category',
                                'supercategory', 'division', 'brand', 'manufacturer','images_url', 'channel_name']].copy()

In [27]:
tabela1.rename(columns = {'network':'CONCORRENTE','city':'CIDADE', 'leaflet_name':'ENCARTE', 'leaflet_type':'TIPO','leaflet_number_of_pages':'PAGINA',
                       'validity_start_date':'INICIO','validity_finish_date':'FIM', 'period':'PERIODO', 'item_description':'DESCRICAO',
                       'ean':'COD_BARRAS','price':'PRECO','minimum_quantity':'QTD_MINIMA', 'dynamic':'DINAMICA', 'item_measure':'TAMANHO',
                       'item_unit_measure':'UNIDADE','category':'CATEGORIA', 'supercategory':'SUPERCATEGORIA', 'division':'DIVISAO',
                       'brand':'MARCA', 'manufacturer':'FORNECEDOR','images_url':'URL', 'channel_name':'SEGMENTO'}, inplace=True)

In [29]:
print(tabela1.columns)

Index(['CONCORRENTE', 'CIDADE', 'ENCARTE', 'TIPO', 'PAGINA', 'INICIO', 'FIM',
       'PERIODO', 'DESCRICAO', 'COD_BARRAS', 'PRECO', 'QTD_MINIMA', 'DINAMICA',
       'TAMANHO', 'UNIDADE', 'CATEGORIA', 'SUPERCATEGORIA', 'DIVISAO', 'MARCA',
       'FORNECEDOR', 'URL', 'SEGMENTO'],
      dtype='object')


In [31]:
print(tabela2.columns)

Index(['SEQCAMPANHA', 'CAMPANHA', 'MIDIA', 'INICIO', 'FIM', 'SEQLOJA',
       'SEQOFERTA', 'TIPO_OFERTA', 'SEQPRODUTO', 'DESCRICAO', 'PRECO_VAREJO',
       'PRECO_ATACADO', 'QUANTIDADE_MINIMA', 'PRECO_FIDELIDADE',
       'SEQFORNECEDOR', 'NEGOCIACAO_ESPACO', 'TIPO_CASHBACK', 'CASHBACK_R$',
       'DESCONTO DE %', 'QTD. PAGUE', 'QTD. LEVE', 'COMPRA DE',
       'DESCONTO EM UNIDADES', 'PRECO_REGULAR', 'PÃGINA'],
      dtype='object')


In [33]:
#Reduzindo base Smarket para apenas o necessário (selecionando apenas as colunas nessa posição)
tabela2 = tabela2[['SEQCAMPANHA','CAMPANHA','INICIO', 'FIM', 'SEQLOJA','SEQOFERTA', 'TIPO_OFERTA', 'SEQPRODUTO', 'DESCRICAO', 'PRECO_VAREJO','PRECO_ATACADO',
                   'QUANTIDADE_MINIMA','PRECO_FIDELIDADE','TIPO_CASHBACK','CASHBACK_R$','DESCONTO DE %', 'QTD. PAGUE', 'QTD. LEVE', 'COMPRA DE',
                   'DESCONTO EM UNIDADES','PRECO_REGULAR', 'PÃGINA']].copy()

In [35]:
tabela2.rename(columns = {'SEQPRODUTO':'COD_PRODUTO','CAMPANHA':'ENCARTE','PÃGINA':'PAGINA'}, inplace=True)

In [37]:
print(tabela3.columns)

Index(['ï»¿EMPRESA', 'COD_PRODUTO', 'cidade', 'negocio', 'COD_BARRAS',
       'COD_FAMILIA', 'DSC_PRODUTO', 'NIVEL1', 'NIVEL2', 'NIVEL3', 'NIVEL4',
       'NIVEL5', 'NIVEL6', 'NIVEL7'],
      dtype='object')


In [39]:
tabela3.rename(columns ={'cidade':'CIDADE', 'negocio':'SEGMENTO','DSC_PRODUTO': 'DESCRICAO'}, inplace=True)

In [41]:
print(tabela3.columns)

Index(['ï»¿EMPRESA', 'COD_PRODUTO', 'CIDADE', 'SEGMENTO', 'COD_BARRAS',
       'COD_FAMILIA', 'DESCRICAO', 'NIVEL1', 'NIVEL2', 'NIVEL3', 'NIVEL4',
       'NIVEL5', 'NIVEL6', 'NIVEL7'],
      dtype='object')


In [43]:
#Verificando 
print(f"Tabela 1 - Linhas:{tabela1.shape[0]}, Colunas: {tabela1.shape[1]}")
print(f"Tabela 2 - Linhas:{tabela2.shape[0]}, Colunas: {tabela2.shape[1]}")
print(f"Tabela 3 - Linhas:{tabela3.shape[0]}, Colunas: {tabela3.shape[1]}")

Tabela 1 - Linhas:35275, Colunas: 22
Tabela 2 - Linhas:19372, Colunas: 22
Tabela 3 - Linhas:570522, Colunas: 14


In [45]:
#Padronizando as descrições
tabela1['DSC_NORM'] = tabela1['DESCRICAO'].str.lower()
tabela2['DSC_NORM'] = tabela2['DESCRICAO'].str.lower()
tabela3['DSC_NORM'] = tabela3['DESCRICAO'].str.lower()

In [85]:
#Concatenando todas as descrições pq o TF-IDF só pode ser treinado 1 vez (msm q as bases sejam mt grandes) deixa 1 só e divide depois
descricoes = pd.concat([
    tabela1['DSC_NORM'],
    tabela2['DSC_NORM'],
    tabela3['DSC_NORM']
], ignore_index=True)

In [87]:
#aplicando o TF - IDF (TF- IDF transforma as descrições em vetores numéricos)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(descricoes)

In [89]:
#Garantindo q o TF - IDF tenha o msm número de feature para cada tabela
tfidf_tabela1 = vectorizer.transform(tabela1['DSC_NORM'])
tfidf_tabela2 = vectorizer.transform(tabela2['DSC_NORM'])
tfidf_tabela3 = vectorizer.transform(tabela3['DSC_NORM'])

In [91]:
#calculando a similaridade do cosseno com nearest neighbors para cada uma das tabelas
nn_model2 = NearestNeighbors(n_neighbors=1, metric= 'cosine').fit(tfidf_tabela2)
nn_model3 = NearestNeighbors(n_neighbors=1, metric= 'cosine').fit(tfidf_tabela3)
#n_neighbors=1 significa que vai buscar o item mais parecido para cada descrição
#metric='cosine' usa a similaridade de cosseno uma métrica eficiente para comparar textos

In [95]:
#Encontrando as descrições mais semelhantes para cada item da tabela 1 
_, indices2 = nn_model2.kneighbors(tfidf_tabela1)
_, indices3 = nn_model3.kneighbors(tfidf_tabela1)
#aqui calculamos os itens mais parecidos apenas para a tabela 1

In [97]:
#Relacionando com as outras tabelas
tabela1['relacionamento_tabela2'] = indices2.flatten()
tabela1['relacionamento_tabela3'] = indices3.flatten()
#aqui significa que cada item da tabela 1 tem o indice do item mais parecido entre todas as descrições (incluindo tab.2 e 3)

In [99]:
#Exibir o resultado
print(tabela1[['DSC_NORM','relacionamento_tabela2','relacionamento_tabela3']])

                                 DSC_NORM  relacionamento_tabela2  \
0             espumante salton brut 750ml                     255   
1             espumante salton brut 750ml                     255   
2             espumante salton brut 750ml                     255   
3             espumante salton brut 750ml                     255   
4             espumante salton brut 750ml                     255   
...                                   ...                     ...   
35270  lip tint balm dailus fps 8 com 4ml                       0   
35271  lip tint balm dailus fps 8 com 4ml                       0   
35272  lip tint balm dailus fps 8 com 4ml                       0   
35273  lip tint balm dailus fps 8 com 4ml                       0   
35274  lip tint balm dailus fps 8 com 4ml                       0   

       relacionamento_tabela3  
0                        5452  
1                        5452  
2                        5452  
3                        5452  
4          

In [101]:
# Exportar o resultado para um arquivo Excel
tabela1.to_excel('tabela1_relacionada.xlsx', index=False)
